# Project: Data Wrangling OSM
Map Area:
I have chosen my neighborhood, Capitol Hill, Seattle.

## 1. Auditing OSM data
### Unique Tags
After downloaded the OSM data of my neighborhood, Capitol Hill, I decided to do a count for each unique tag in the OSM file, just to make sure that the data is reasonable. The script used was **mapparser.py**, generating the following output:

* "node": 203843, 
* "member": 24732, 
* "nd": 221071, 
* "tag": 218155, 
* "bounds": 1, 
* "note": 1, 
* "meta": 1, 
* "relation": 563, 
* "way": 22017, 
* "osm": 1

As expected, I am getting a lot of "node" and "way" tags, and that the number of node tags is greater than that of the way tags. This is reasonable becuase ways are just a series of connected nodes.  

### Check characters in Tags
Each tag "tag" has a "k" value indicating information that the tag contains. The script **tags.py** uses regular expressions to sort and count the "k" values of the tags. 
Here are the sorting criteria for the four categories:
* **problemchars**: has at least one of the problem characters
* **lower **: consists of only lowercase alphabets
* **lower_colon**: consists of lowercase alphabets followed by one colon character and lowercase alphabets
* **other**: anything else that doesn't fit those three categories

The following is the count for each category:

* "problemchars": 0, 
* "lower": 110036, 
* "lower_colon": 103595
* "other": 4524, 

It is good that we do not have any tags with problem characters in their "k" values. Most "k" values just consists of alphabets, and many of them has a colon with alphabets, suggesting that we will need to parse those with colon later on when building a heirarchical database.

## 2. Problems encountered in my map
### Street name inconsistency
The script **audit.py** identifies odd street names and shows the converted names for each identified "odd street name":
* 12th Ave E =>  12th Avenue E
* 20th Ave E =>  20th Avenue E
* E Union St. =>  E Union Street
* Yesler =>  Yesler
* Westlake Ave =>  Westlake Avenue
* 18th Ave =>  18th Avenue
* 4th Ave =>  4th Avenue
* 2nd Ave =>  2nd Avenue
* 3rd Ave =>  3rd Avenue
* 34th Ave =>  34th Avenue
* 1st Ave =>  1st Avenue
* Boylston Ave =>  Boylston Avenue
* 14th Ave =>  14th Avenue
* Broadway =>  Broadway
* Broadway East =>  Broadway East



In summary the following street name abbreviations require corrections:
* Ave => Avenue
* St. = > Street

The following street names are exceptions that do not require corrections:
* Yesler
* Broadway
* Broadway East

### Implementing changes to street names
The python script **data.py** parses the OSM file into csv files.

The bash script **clean_street_names.sh** performs text substitution on nodes_tags.csv and ways_tags.csv and converts the above mentioned odd street names to their corrected ones.

## 3. Overview of the data

### 3.1 File sizes

* CustomSeattle.osm: 51.7MB
* CustomSeattle.db: 29.3MB
* nodes.csv: 17.3MB
* nodes_tags.csv: 4.1MB
* ways.csv: 1.3MB
* ways_nodes.csv: 5.3MB
* ways_tags/csv" 4.1MB



### Overview of statistics of the dataset:



### 3.2 Number of unique users

**Query**:

    SELECT COUNT(DISTINCT uid) FROM 
        (SELECT N.uid FROM nodes AS N
            UNION
        SELECT W.uid FROM ways AS W);
**Output**:
    428

### 3.3 Number of nodes and ways

**Query for nodes**:

    SELECT COUNT(DISTINCT id) FROM nodes;
**Output, number of nodes**:

    203843

**Query for ways**:

    SELECT COUNT(DISTINCT id) FROM ways;
**Output, number of ways**:

    22017


### 3.4 Number of restaurants

**Query**:

    SELECT COUNT(*) FROM nodes_tags
    WHERE key = "amenity"
    AND value = "restaurant";
    
**Output**:

    388

### 3.5 Top 5 highest counts of type of cuisine served in restaurants

**Query**:

    SELECT value, COUNT(*) AS count 
    FROM nodes_tags
    WHERE id IN (
        SELECT id FROM nodes_tags
        WHERE key = "amenity"
        AND value = "restaurant"
        )
    AND key = "cuisine"
    GROUP BY value
    ORDER BY count DESC
    LIMIT 5;
    
**Output**:  
mexican|24  
italian|23  
pizza|21  
thai|19  
japanese|17  
 




### 3.6 Top 5 roads with the most nodes

**Query**:

    SELECT value, count 
    FROM ( 
        SELECT id, COUNT(*) AS count
        FROM ways_nodes
        GROUP BY id
        ) AS wn
    JOIN ways_tags AS w
    ON wn.id = w.id
    WHERE key = "street"
    ORDER BY count DESC
    LIMIT 5;
    
**Output**:  
4th Avenue North|147  
East Yesler Way|125  
John Street|106  
Wall Street|101  
East Yesler Way|93  

 




### 3.7 Top 5 contributors to the bike racks

**Query**:
    
    SELECT user, COUNT(*) AS count
    FROM nodes    
    WHERE nodes.id IN 
        (SELECT id FROM nodes_tags 
        WHERE key LIKE "bike_rack%"
        GROUP BY id)
    GROUP BY uid
    ORDER BY count DESC
    LIMIT 5;
   
    
    
    
    
**Output**:  
WBSKI|631  
sctrojan79|150  
rza31|38  
bdp|33  
Omnific|28  


## 4. Other ideas about the dataset
### 4.1 Find the best place to live in Capitol Hill
The best place to live in Capitol Hill can be found by minimizing the following objective function (i.e. minimize cost of living):

Cost of living = Cost to walk to work + Cost of losing sleep

Where:
* Cost to walk to work is proportional to the distance from the Oddfellows Cafe (lon, lat = 47.6148943,-122.321752517) to the apartment of interest
* Cost of losing sleep is a function of noise from bars, which is approximately inversely proportional to the squared distance from the nearest bar to the apartment of interest

Since both metrics require the computation of distances, I decide to use pysqlilte, so that I can customize a DIST() function for returning distances given a pair of lon and lat.

The following briefly walks through the Python script. For details of each SQL query, please refer to the Python script.



#### Getting coordinates of all apartments in the area:
The apartments are usually large buildings; the ways_nodes table outlines the boundnary of these building with a series of nodes. I decided to average the lon and lat of the series of nodes of each apartment building to get a single point location for each apartment. Unfortunately, some of apartments were stored in the 'nodes' tables instead of the 'ways' tables. Therefore I had to use "UNION" to combine the results together.

**Query:**

    CREATE VIEW apt_pos AS
		SELECT n.id, n.lat, n.lon, nt2.value AS name
		FROM nodes_tags AS nt
		JOIN nodes_tags AS nt2
		ON nt.id = nt2.id
		JOIN nodes AS n
			ON n.id = nt.id
		WHERE 
			(nt.key = 'building' OR  nt.value = 'residential')
			AND nt.value LIKE 'apartment%'
			AND nt2.key = 'name'
			
			UNION

		SELECT wt.id, AVG(n.lat) AS lat, 
			AVG(n.lon) AS lon, wt2.value AS name
		FROM ways_tags AS wt
		JOIN ways_tags AS wt2
            ON wt.id = wt2.id	
		JOIN ways_nodes AS wn
            ON wt.id = wn.id
		JOIN nodes AS n
			ON n.id = wn.node_id
		WHERE 
			(wt.key = 'building' OR  wt.value = 'residential')
			AND wt.value LIKE 'apartment%'
			AND wt2.key = 'name'
		GROUP BY wt.id;

**Truncated Output of this Temp View apt_pos**:

444092514,47.6258002818,-122.344453945,708 Uptown  
456138164,47.6176869917,-122.304130758,Session  
490586222,47.6171625944,-122.337787544,Stratus  
492248771,47.6176266435,-122.339369665,McKenzie Apartments  
492680050,47.6216989714,-122.338885314,The Lofts


#### Getting coordinates of all bars in the area:
A similar process was done to get all coordinates of the bars in the area (please refer to the Python script for details).

**Query**:

	CREATE VIEW bar_pos AS          
		SELECT n.id, nt2.value AS name, n.lat, n.lon 
		FROM nodes_tags AS nt1
		JOIN nodes_tags AS nt2 
			ON nt1.id = nt2.id
		JOIN nodes AS n
			ON n.id = nt1.id
		WHERE nt1.key = 'amenity' 
			AND nt1.value = 'bar'
			AND nt2.key = 'name';
**Truncated Output of Table bar_pos**:  
4522428050|Cantine|47.6237717|-122.3388101  
4527603992|Yeti Bar|47.6185987|-122.3210229  
4531209989|Fountain Wine Bar & Lounge|47.6110689|-122.3340032  
4588626389|Foreign National|47.6142159|-122.3274435  
4677585271|mbar|47.6227869|-122.3338623  


#### For each apartment, calculate distances to the nearest bar, and to work

**Query**:

	CREATE TEMP VIEW A_B_dists AS
	SELECT A.id, A.lat, A.lon, A.name, B.name,
		MIN(dist(A.lat, A.lon, B.lat, B.lon)) AS b_dist,
		dist(A.lat, A.lon, 47.6148943, -122.321752517) 
        AS w_dist
	FROM apt_pos AS A
	JOIN bar_pos AS B
	GROUP BY A.id;

**Truncated Output of Table A_B_dists**:

44598313,47.6132006857,-122.332295986,Premiere on Pine,Fountain Wine Bar & Lounge,0.180419920635,0.653316433256  
68665991,47.611038375,-122.341193633,Viktoria,Nitelite,0.0335220834781,1.21541106455  
118979624,47.61219518,-122.329718968,Elektra Condominiums,Still Liquor,0.188285581865,0.520351253027  
140676009,47.609754675,-122.325954112,The Chasselton,Sun Liquor Distillery,0.318226734371,0.438004181541  
140676014,47.6090702,-122.32919012,Emerson Apartments,Regatta Bar & Grille,0.183395353659,0.606615541837


#### Convert distances to costs and find the best apartment!
A scoring function was defined in Python and used in the SQL query.
Using the following expression, I can convert distances to costs:
* Daily cost of walking to work = 2 \* distance to work \* 1hr/3miles * $35/hr  

* For calculating daily cost of losing sleep:  
    estimate noise from the nearest bar = 90 decibels \* (0.00568182 miles / distance to closest bar)^2  
    if estimated noise from the nearest bar <= 60 decibels:  
    then the cost = 0  
    else:  
    Daily cost of losing sleep = \$5/decibel \* (estimated noise - 60 decibels) + \$50  



**Query**:  
	
    SELECT A_name,
		scoring(b_dist, w_dist) AS score
	FROM A_B_dists
	ORDER BY score
    
**First 5 Output**:

Cue Apartments,0.894976218219  
Pike Motorworks,1.76264398828  
The Starbird,1.98814250979  
The Lenawee,2.22456104679  
Glencoe,2.84915844525  


Based on my two metrics (distance to work and to nearest bars), I have found the best apartment, which is Cue Apartments!!!!!